In [18]:
import pandas as pd 
import numpy as np 
import  matplotlib.pyplot as plt 
import pickle
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings 
import ast
import faiss

In [11]:
df = pd.read_csv("D:\\MCP SERVER PROJECTS\\Receipe_Recommendation\\recipe_final (1).csv")

In [12]:
df

,Unnamed: 0,recipe_id,recipe_name,aver_rate,image_url,review_nums,calories,fat,carbohydrates,protein,cholesterol,sodium,fiber,ingredients_list
0,0,222388,Homemade Bacon,5.00,https://images.media-allrecipes.com/userphotos...,3,15,36,1,42,21,81,2,"['pork belly', 'smoked paprika', 'kosher salt'..."
1,1,240488,"Pork Loin, Apples, and Sauerkraut",4.76,https://images.media-allrecipes.com/userphotos...,29,19,18,10,73,33,104,41,"['sauerkraut drained', 'Granny Smith apples sl..."
2,2,218939,Foolproof Rosemary Chicken Wings,4.57,https://images.media-allrecipes.com/userphotos...,12,17,36,2,48,24,31,4,"['chicken wings', 'sprigs rosemary', 'head gar..."
3,3,87211,Chicken Pesto Paninis,4.62,https://images.media-allrecipes.com/userphotos...,163,32,45,20,65,20,43,18,"['focaccia bread quartered', 'prepared basil p..."
4,4,245714,Potato Bacon Pizza,4.50,https://images.media-allrecipes.com/userphotos...,2,8,12,5,14,7,8,3,"['red potatoes', 'strips bacon', 'Sauce:', 'he..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48730,48730,222886,Grateful Dead Cocktail,3.50,https://images.media-allrecipes.com/userphotos...,4,20,1,6,1,0,1,0,"['fluid ounce tequila', 'fluid ounce vodka', '..."
48731,48731,25650,Cheese Filling For Pastries,4.33,https://images.media-allrecipes.com/userphotos...,3,6,14,2,4,13,3,1,"['raisins', 'brandy', 'cream cheese', 'white s..."
48732,48732,23544,Peach Smoothie,3.62,https://images.media-allrecipes.com/userphotos...,21,8,7,8,10,3,3,8,"['sliced peaches drained', 'scoops vanilla ice..."
48733,48733,170710,Double Dare Peaches,4.71,https://images.media-allrecipes.com/userphotos...,19,20,33,16,11,25,7,5,"['butter', 'habanero peppers', 'fresh peaches'..."


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48735 entries, 0 to 48734
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        48735 non-null  int64  
 1   recipe_id         48735 non-null  int64  
 2   recipe_name       48735 non-null  object 
 3   aver_rate         48735 non-null  float64
 4   image_url         48735 non-null  object 
 5   review_nums       48735 non-null  int64  
 6   calories          48735 non-null  int64  
 7   fat               48735 non-null  int64  
 8   carbohydrates     48735 non-null  int64  
 9   protein           48735 non-null  int64  
 10  cholesterol       48735 non-null  int64  
 11  sodium            48735 non-null  int64  
 12  fiber             48735 non-null  int64  
 13  ingredients_list  48735 non-null  object 
dtypes: float64(1), int64(10), object(3)
memory usage: 5.2+ MB


In [14]:
df["ingredients_list"]=df["ingredients_list"].apply(lambda x :ast.literal_eval(x))


In [15]:
df["ingredients_list"]= df["ingredients_list"].apply(lambda x:" ".join(x))

In [16]:
Model = NVIDIAEmbeddings(model="nvidia/nv-embedqa-e5-v5",model_type ="passage",
    nvidia_api_key="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"
)

In [17]:
receipe_embedding =[]
batch_size = 10

for i in range(0,len(df),batch_size):
    batch_text=df["ingredients_list"].iloc[i:i+batch_size].tolist()
    embeding_batch = Model.embed_documents(batch_text)
    receipe_embedding.extend(embeding_batch)

embeddings = np.array(receipe_embedding)

In [21]:

np.save("receipe_embeddings.npy", embeddings)
df.to_pickle("receipe.pkl")


dimension = embeddings.shape[1]

index = faiss.IndexFlatIP(dimension) 


index.add(embeddings)

faiss.write_index(index, "receipe_faiss.index")

print("FAISS index built and saved as receipe_faiss.index.")

FAISS index built and saved as receipe_faiss.index.
